In [ ]:
import tensorflow as tf
import numpy as np
import gym
import collections
import random
import os

In [2]:
ffmpeg_path = 'C:\\Users\\dhzns\\OneDrive\\code\\ipython\\OpenAI_gym\\ffmpeg\\bin'
os.environ['PATH'] = ffmpeg_path + ';' + os.environ['PATH']

In [3]:
#File Path
filepath_ckpt  = "./ckpt/CartPole_yt.ckpt" #weight saver check point file path
batch_size = 40
learning_rate = 0.01
input_size = 4
output_size = 2
h_size = 128
discount_rate = 0.85

In [4]:
class DQN:
    def __init__(self, session, input_size, output_size, name="main"):
        self.session = session
        self.input_size = input_size
        self.output_size = output_size
        self.net_name = name
        
        self._build_network(h_size = h_size, l_rate = learning_rate)
        
    def _build_network(self, h_size=10, l_rate=0.05):
        with tf.name_scope(self.net_name) as scope:
            self._X = tf.placeholder(tf.float32, [None, input_size], name='Observation_placeholder')
            self._Y = tf.placeholder(tf.float32, [None, output_size], name='Q_placeholder')

            W1 = tf.Variable(tf.truncated_normal([self.input_size, h_size], dtype=tf.float32), trainable=True, name='weight')
            l1p = tf.matmul(self._X,W1)
            l1 = tf.nn.tanh(l1p)
        
#             tf.contrib.layers.batch_norm(l1)
            W2 = tf.Variable(tf.truncated_normal([h_size, output_size], dtype=tf.float32), trainable=True, name='weight')
            l2p = tf.matmul(l1, W2)
#             l2 = tf.nn.relu(l2p)
      
#             tf.contrib.layers.batch_norm(l2)
#             W3 = tf.Variable(tf.truncated_normal([h_size, h_size], dtype=tf.float32), trainable=True, name='weight')
#             l3p = tf.matmul(l2, W3)
#             l3 = tf.nn.relu(l3p)
        
#             tf.contrib.layers.batch_norm(l3)
#             W4 = tf.Variable(tf.truncated_normal([h_size, self.output_size], dtype=tf.float32), trainable=True, name='weight')
#             l4p = tf.matmul(l3, W4)
#             l4 = tf.nn.relu(l4p)
            
            self.logits = l2p
#             self.prob = tf.nn.tanh(self.logits)

            self._loss = tf.reduce_sum(tf.square(self._Y - self.logits))
            self._train = tf.train.AdamOptimizer(learning_rate=l_rate).minimize(self._loss)
            
    def predict(self, state):
        x = np.reshape(state, [1, self.input_size])
        return self.session.run(self.logits, feed_dict={self._X: x})
    
    def update(self, x_stack, y_stack):
        return self.session.run([self._loss, self._train], feed_dict={self._X: x_stack, self._Y: y_stack})

In [5]:
def replay_train(mainDQN, targetDQN, train_batch):
    x_stack = np.empty(0).reshape(0, input_size)
    y_stack = np.empty(0).reshape(0, output_size)
    
    for state, action, reward, next_state, done in train_batch:
        Q = mainDQN.predict(state)
        
        if done:
            Q[0, action]= reward
        else:
            Q[0, action]= reward + discount_rate * np.max(targetDQN.predict(next_state))
        
        y_stack = np.vstack([y_stack, Q])
        x_stack = np.vstack([x_stack, state])
        
    return mainDQN.update(x_stack, y_stack)

In [6]:
env = gym.make('CartPole-v0')
env = gym.wrappers.Monitor(env, './tmp/cartpole-v0-experiment2', force=True )
max_episode = 2000
replay_buffer = collections.deque()
REPLAY_MEMORY = 4000

[2017-08-30 20:43:39,812] Making new env: CartPole-v0
[2017-08-30 20:43:39,833] Clearing 18 monitor files from previous run (because force=True was provided)


In [7]:
sess = tf.Session()

In [8]:
mainDQN = DQN(sess, input_size, output_size, name="main")
targetDQN = DQN(sess, input_size, output_size, name="target")

saver = tf.train.Saver()
sess.run(tf.global_variables_initializer())

# saver.restore(sess, filepath_ckpt)

In [9]:
def get_copy_var_ops(*, dest_scope_name="target", src_scope_name="main"):
    op_holder = []
    
    src_vars = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, scope=src_scope_name)
    dest_vars = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, scope=dest_scope_name)
    
    for src_var, dest_var in zip(src_vars, dest_vars):
        op_holder.append(dest_var.assign(src_var.value()))
        
    return op_holder

In [10]:
copy_ops = get_copy_var_ops(dest_scope_name="target", src_scope_name = "main")
sess.run(copy_ops)
print()

In [ ]:
p= 0.2
gamma = 0.8
for i_episode in range(2001):
    if p > 0:
        p -=0.01
    s = env.reset()
    for t in range(201):
#         env.render()
        e = random.random()
#         print(e)
        if e < p:
            a = env.action_space.sample()
        else:
#             print("select")
            a = np.argmax(mainDQN.predict(s))
#         print(action)
        
        #get new state    
        s1, rwd, done, _ = env.step(a)
        
        if t == 199:
            replay_buffer.append((s, a, rwd, s1, done))
            if len(replay_buffer) > REPLAY_MEMORY:
                replay_buffer.popleft()
            break
        if done:
            if t <10:
                rwd = -200
            elif t< 50:
                rwd = -150
            elif t< 100:
                rwd = -100
            elif t< 150:
                rwd = -50
        replay_buffer.append((s, a, rwd, s1, done))
        if len(replay_buffer) > REPLAY_MEMORY:
            replay_buffer.popleft()
        
        if done:
            break
            
        s = s1
        
    
            
        
    if i_episode % 2 == 0: 
        print("Episode finished after {} timesteps".format(t+1))

        for _ in range(50):
            mini_batch = random.sample(replay_buffer, batch_size)
            loss, _ = replay_train(mainDQN, targetDQN, mini_batch)
        print("Loss : ", loss)
        sess.run(copy_ops)

[2017-08-30 20:43:42,852] Starting new video recorder writing to C:\Users\dhzns\OneDrive\code\ipython\OpenAI_gym\tmp\cartpole-v0-experiment2\openaigym.video.0.6608.video000000.mp4


Episode finished after 170 timesteps


[2017-08-30 20:43:47,755] Starting new video recorder writing to C:\Users\dhzns\OneDrive\code\ipython\OpenAI_gym\tmp\cartpole-v0-experiment2\openaigym.video.0.6608.video000001.mp4


Loss :  21.6852
Episode finished after 78 timesteps
Loss :  120.549
Episode finished after 41 timesteps
Loss :  5985.17
Episode finished after 200 timesteps


[2017-08-30 20:43:50,643] Starting new video recorder writing to C:\Users\dhzns\OneDrive\code\ipython\OpenAI_gym\tmp\cartpole-v0-experiment2\openaigym.video.0.6608.video000008.mp4


...

[2017-08-30 20:47:47,152] Starting new video recorder writing to C:\Users\dhzns\OneDrive\code\ipython\OpenAI_gym\tmp\cartpole-v0-experiment2\openaigym.video.0.6608.video000729.mp4


Loss :  169.576
Episode finished after 200 timesteps
Loss :  90.7702
Episode finished after 200 timesteps
Loss :  74.5437
Episode finished after 200 timesteps
Loss :  113.081
Episode finished after 200 timesteps
Loss :  102.54
Episode finished after 200 timesteps
Loss :  171.429
Episode finished after 200 timesteps
Loss :  48.3336
Episode finished after 200 timesteps
Loss :  117.381
Episode finished after 200 timesteps
Loss :  49.0895
Episode finished after 200 timesteps
Loss :  76.1379
Episode finished after 200 timesteps
Loss :  59.94
Episode finished after 200 timesteps
Loss :  87.3425
Episode finished after 200 timesteps
Loss :  137.35
Episode finished after 200 timesteps
Loss :  69.4938
Episode finished after 200 timesteps
Loss :  110.472
Episode finished after 200 timesteps
Loss :  48.7776
Episode finished after 200 timesteps
Loss :  163.643
Episode finished after 200 timesteps
Loss :  133.254
Episode finished after 200 timesteps
Loss :  94.6749
Episode finished after 200 timeste

[2017-08-30 20:49:08,430] Starting new video recorder writing to C:\Users\dhzns\OneDrive\code\ipython\OpenAI_gym\tmp\cartpole-v0-experiment2\openaigym.video.0.6608.video001000.mp4


Loss :  51.9393
Episode finished after 200 timesteps
Loss :  86.7996
Episode finished after 200 timesteps
Loss :  49.0388
Episode finished after 200 timesteps
Loss :  106.6
Episode finished after 200 timesteps
Loss :  65.7616
Episode finished after 200 timesteps
Loss :  62.5569
Episode finished after 200 timesteps
Loss :  118.428
Episode finished after 200 timesteps
Loss :  84.168
Episode finished after 200 timesteps
Loss :  64.035
Episode finished after 200 timesteps
Loss :  60.8896
Episode finished after 200 timesteps
Loss :  84.3533
Episode finished after 200 timesteps
Loss :  51.685
Episode finished after 200 timesteps
Loss :  37.4663
Episode finished after 200 timesteps
Loss :  42.0764
Episode finished after 200 timesteps
Loss :  95.4814
Episode finished after 200 timesteps
Loss :  34.3706
Episode finished after 200 timesteps
Loss :  25.4419
Episode finished after 200 timesteps
Loss :  72.1752
Episode finished after 200 timesteps
Loss :  28.0594
Episode finished after 200 timestep

Loss :  79.0343
Episode finished after 88 timesteps
Loss :  23.4899
Episode finished after 124 timesteps
Loss :  74.633
Episode finished after 200 timesteps
Loss :  73.6762
Episode finished after 200 timesteps
Loss :  70.7623
Episode finished after 200 timesteps
Loss :  94.4038
Episode finished after 188 timesteps
Loss :  85.5386
Episode finished after 200 timesteps
Loss :  56.1293
Episode finished after 200 timesteps
Loss :  57.7452
Episode finished after 169 timesteps
Loss :  89.4638
Episode finished after 200 timesteps
Loss :  124.622
Episode finished after 105 timesteps
Loss :  37.5038
Episode finished after 189 timesteps
Loss :  47.2104
Episode finished after 200 timesteps
Loss :  79.145
Episode finished after 200 timesteps
Loss :  58.4493
Episode finished after 189 timesteps
Loss :  38.21
Episode finished after 200 timesteps
Loss :  45.0763
Episode finished after 116 timesteps
Loss :  59.4141
Episode finished after 199 timesteps
Loss :  93.5754
Episode finished after 200 timestep

In [ ]:
saver.save(sess, filepath_ckpt)

In [ ]:
env.close()

In [ ]:
gym.upload('./tmp/cartpole-v0-experiment2', api_key='sk_R6pSeGWSRfSWyCxZP08FiQ')

In [ ]:
# for the test
env = gym.make('CartPole-v0')
for i_episode in range(20):
    s = env.reset()
    for t in range(201):
        env.render()
        a = np.argmax(mainDQN.predict(s))
        s, rwd, done, _ = env.step(a)
        
        if done:
            print("Episode finished after {} timesteps".format(t+1))
            break

In [ ]:
env.close()
sess.close()